In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [26]:
class Thermo():
    
    def __init__(self, ID):
        
        seed = {}
        self.ID = ID
        seed['Molecular Weight'] = []
        seed['InChI Code'] = []
        seed['InChIKey Code'] = []
        seed['CAS Reg. #'] = []
        seed['TP Temp (K)'] = []
        
        df = pd.DataFrame(data=seed)
        print(self.ID)
        self.data = df

    def nist_pull(self):
        page='https://webbook.nist.gov/cgi/cbook.cgi?ID=' + self.ID + '&Units=SI&Mask=4#Thermo-Phase'
        #https://webbook.nist.gov/cgi/cbook.cgi?ID=C7782447&Units=SI&Mask=4#Thermo-Phase
        #'https://webbook.nist.gov/cgi/cbook.cgi?ID=C7727379&Units=SI&Mask=4#Thermo-Phase'

        # Update so chemical id is the input

        # Different behavior for first entry versus following entries
        driver = webdriver.Chrome()

        driver.get(page)
        content = driver.page_source
        soup = BeautifulSoup(content, 'html.parser')

        data = {}
        for i, a in enumerate(soup.find('main').find('ul').find_all('li')):
            #print('{}: {}'.format(i, a))
            if i == 1:
                data['Molecular Weight'] = [str(a).split('</strong> ')[1].split('<')[0]]
            if i == 2:
                data['InChI Code'] = [str(a).split('monospace;">')[1].split('</span></li>')[0]]
            if i == 5:
                data['InChIKey Code'] = [str(a).split('font-family: monospace;">')[1].split('</span></li>')[0]]
            if i == 6:
                data['CAS Reg. #'] = [str(a).split('</strong> ')[1].split('</li>')[0]]

        # Need to modify this section to be generalizable to all pages.
        # See hanges in beautifulsoup_debug
        # Also add in try/except statements if they don't exist.
        test_data = str(soup.find('table', attrs = {'aria-label': 'One dimensional data'}).find('tbody'))
        
        try:
            data['TP Temp (K)'] = (test_data.split('T<sub>triple</sub></td><td class="right-nowrap">')[1].split('</td>')[0])
            
        except:
            data['TP Temp (K)'] = ('--')
            
        try:
            data['TP Pres (bar)'] = (test_data.split('P<sub>triple</sub></td><td class="right-nowrap">')[1].split('</td>')[0])
            
        except:
            data['TP Pres (bar)'] = ('--')
            
        try:
            data['CP Temp (K)'] = (test_data.split('T<sub>c</sub></td><td class="right-nowrap">')[1].split('</td>')[0])
            
        except:
            data['CP Temp (K)'] = ('--')
        
        try:
            data['CP Pres (bar)'] = (test_data.split('P<sub>c</sub></td><td class="right-nowrap">')[1].split('</td>')[0])
            
        except:
            data['CP Pres (bar)'] = ('--')
        

        driver.quit()

        self.data = self.data.append(data, ignore_index=True)
        #return data
        
        # Add in VLE functions

In [27]:
d = Thermo("C7727379")

C7727379


NameError: name 'df' is not defined